# **配当割引モデルに基づく理論株価の計算**

## **手順1: 資本コスト推定**

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# トヨタのティッカーシンボル
ticker = "7203.T"
# 日経225のティッカーシンボル
index_ticker = "^N225"

# 日付の範囲を設定（過去3年間）
end_date = datetime.strptime('2024-07-26', '%Y-%m-%d') #datetime.now()
start_date = end_date - timedelta(days=3*365)

# データを取得
toyota_data = yf.download(ticker, start=start_date, end=end_date)
index_data = yf.download(index_ticker, start=start_date, end=end_date)

# 週次収益率を計算
toyota_weekly = toyota_data['Adj Close'].resample('W').last().pct_change().dropna()
index_weekly = index_data['Adj Close'].resample('W').last().pct_change().dropna()

# データを表示
print(toyota_weekly.head())
print(index_weekly.head())

# βの計算
cov_matrix = np.cov(toyota_weekly, index_weekly)
beta = cov_matrix[0, 1] / cov_matrix[1, 1]

# 資本コストの計算（CAPMモデル）
risk_free_rate = 0.0084  # 10年国債利回りの長期平均値
market_premium = 0.05  # リスクプレミアム
cost_of_equity = risk_free_rate + beta * market_premium

print(f"推定されたβ: {beta:.4f}")
print(f"推定された資本コスト: {cost_of_equity:.4f}")

## **手順2: 配当の成長率g1の計算**

In [ ]:
# トヨタのティッカーシンボルで財務データを取得
toyota = yf.Ticker(ticker)

# 配当履歴を取得
dividends = toyota.dividends

# 過去6年間の配当データを抽出
dividends_last_6_years = dividends[-6*2:]  # 過去6年分の半期データ

# 過去6年間の配当データを表示
print("過去6年間の配当データ:")
print(dividends_last_6_years)

# 年次配当を計算する関数
def calculate_annual_dividends(dividends):
    """
    各年の9月から翌年の3月までの配当を合計して、年次配当を計算する。

    パラメータ:
    dividends (pd.Series): 配当履歴のシリーズ（日付インデックス付き）

    戻り値:
    pd.Series: 各年の年次配当
    """
    annual_dividends = {}
    for year in range(dividends.index.year.min(), dividends.index.year.max()):
        try:
            # 9月と翌年3月の配当を合計
            sep_dividend = dividends[f'{year}-09']
            mar_dividend = dividends[f'{year+1}-03']
            annual_dividends[year+1] = sep_dividend.sum() + mar_dividend.sum()
        except KeyError:
            # データがない場合はスキップ
            continue

    # 年次配当をシリーズとして返す
    return pd.Series(annual_dividends)

# 年次配当を計算
annual_dividends = calculate_annual_dividends(dividends_last_6_years)

print("年次配当:")
print(annual_dividends)

# 過去5年間の配当成長率の計算
div_growth_rates = annual_dividends.pct_change().dropna()

print("配当成長率:")
print(div_growth_rates)

g1_div_growth = div_growth_rates.mean()

print(f"過去5年間の平均配当成長率: {g1_div_growth:.4f}")

## 手順3: **理論株価の計算**

In [ ]:
def calculate_ddm_two_stage(D0, g1, r, n=5):
    """
    2段階の配当割引モデルを使用して株式の理論価値を計算する関数。

    パラメータ:
    D0 (float): 初年度の配当金
    g1 (float): 最初の５年間の成長率
    r (float): 割引率
    n (int): 初段階の期間（年数）

    戻り値:
    float: 株式の理論価値
    """
    # 最初のn年間の配当の現在価値を計算
    divs_present_value = 0
    for t in range(1, n + 1):
        divs_present_value += D0 * (1 + g1)**t / (1 + r)**t

    # n年目の配当の計算
    Dn = D0 * (1 + g1)**n

    # n年目以降の配当の現在価値（0%成長仮定）
    terminal_value = 1 / (1 + r)**n * Dn / r

    # 合計現在価値
    value = divs_present_value + terminal_value

    return value

# 初年度の配当金（最新の年次配当）
D0 = annual_dividends.loc[2024]

# 成長率g1は、過去5年間の平均配当成長率を使用
g1 = g1_div_growth

# 理論株価の計算
stock_value = calculate_ddm_two_stage(D0, g1, cost_of_equity)
print(f"トヨタの理論株価は: ¥{stock_value:.2f}")

# 最新の株価を取得
latest_price = toyota_data['Adj Close'].iloc[-1] #列の最後の値を取得
print(f"トヨタの最新の株価は: ¥{latest_price:.2f}")

# **残余利益モデルに基づく理論株価の計算**

## **手順1: ROE, 純資産成長率g1の計算**

In [ ]:
import yfinance as yf
import pandas as pd

# トヨタのティッカーシンボル
ticker = "7203.T"

# トヨタのデータを取得
toyota = yf.Ticker(ticker)

# 損益計算書（EPS）とバランスシート（BPS）を取得
income_statement = toyota.financials.T
balance_sheet = toyota.balance_sheet.T

# 株式の発行済み株式数を取得
shares_outstanding = toyota.info['sharesOutstanding']

# EPSの計算 (Net Income / Shares Outstanding)
income_statement['EPS'] = income_statement['Net Income'] / shares_outstanding

# BPSの計算 (Common Stock Equity / Shares Outstanding)
balance_sheet['BPS'] = balance_sheet['Common Stock Equity'] / shares_outstanding

# 配当履歴（DPS）を取得
dividends = toyota.dividends

# 年次配当を計算する関数
def calculate_annual_dividends(dividends):
    """
    各年の9月から翌年の3月までの配当を合計して、年次配当を計算する。

    パラメータ:
    dividends (pd.Series): 配当履歴のシリーズ（日付インデックス付き）

    戻り値:
    pd.Series: 各年の年次配当（最新の年が最初）
    """
    annual_dividends = {}
    for year in range(dividends.index.year.min(), dividends.index.year.max()):
        try:
            # 9月と翌年3月の配当を合計
            sep_dividend = dividends[f'{year}-09']
            mar_dividend = dividends[f'{year + 1}-03']
            annual_dividends[year+1] = sep_dividend.sum() + mar_dividend.sum()
        except KeyError:
            # データがない場合はスキップ
            continue

    # 年次配当をシリーズとして返す（最新の年が最初に来るようにソート）
    annual_dividends_series = pd.Series(annual_dividends)
    return annual_dividends_series.sort_index(ascending=False)

# 年次配当（DPS）を計算
annual_dps = calculate_annual_dividends(dividends)
annual_dps.name = 'DPS'

print("計算された年次配当（DPS）:")
print(annual_dps[:8])

# 年度末の日付を使用してデータフレームを作成
eps_data = income_statement[['EPS']]
bps_data = balance_sheet[['BPS']]

# 年度をインデックスとして設定
eps_data.index = pd.to_datetime(eps_data.index).year
bps_data.index = pd.to_datetime(bps_data.index).year
annual_dps.index = annual_dps.index

# 日付をキーとしてデータを結合
combined_data = pd.concat([eps_data, bps_data, annual_dps], axis=1, join='inner').dropna()

print("\n結合されたデータ（EPS、BPS、DPS）:")
print(combined_data)

# 過去4年間のデータを抽出
data = combined_data.iloc[:4].copy()  # 最新の4年間のデータのコピーを作成

# ROEと配当性向の計算
data = data.assign(
    ROE=data['EPS'] / data['BPS'],
    配当性向=data['DPS'] / data['EPS']
)

print("\n過去4年間の計算されたデータ:")
print(data)

# 平均ROEと配当性向の計算
average_roe = data['ROE'].mean()
average_payout_ratio = data['配当性向'].mean()

print(f"\n過去4年間の平均ROE: {average_roe:.4f}")
print(f"過去4年間の平均配当性向: {average_payout_ratio:.4f}")

# サステイナブル成長率g1の計算
g1 = average_roe * (1 - average_payout_ratio)
print(f"サステイナブル成長率g1: {g1:.4f}")

# 資本コスト（計算済みの値を使用）
print(f"資本コスト: {cost_of_equity:.4f}")


## 手順2: 理論価格の計算

In [ ]:
# 残余利益モデルを用いた理論株価の計算
def calculate_rim(book_value, roe, g1, r, n=5):
    """
    残余利益モデルを使用して株式の理論価値を計算する関数。

    パラメータ:
    book_value (float): 現在の株主資本（簿価）
    roe (float): 最初の5年間の平均ROE
    g1 (float): 最初の5年間のサステイナブル成長率
    r (float): 資本コスト（要求収益率）
    n (int): 初段階の期間（年数）

    戻り値:
    float: 株式の理論価値
    """
    # 残余利益の計算と現在価値
    residual_income_present_value = 0
    current_book_value = book_value

    for t in range(1, n + 1):
        residual_income = (roe - r) * current_book_value
        residual_income_present_value += residual_income / (1 + r)**t

        # Book Valueの成長を考慮
        current_book_value *= (1 + g1)

    # n年目以降の残余利益の現在価値
    terminal_value = (roe - r) * current_book_value / r / (1 + r)**n

    # 合計現在価値
    value = book_value + residual_income_present_value + terminal_value

    return value

# 現在の株主資本（最新のBPS）
book_value = combined_data['BPS'].iloc[0]
print(f"\n現在のBPS: ¥{book_value:.2f}")

# 理論株価の計算
stock_value = calculate_rim(book_value, average_roe, g1, cost_of_equity)
print(f"理論株価: ¥{stock_value:.2f}")

# 最新の株価を取得
latest_price = yf.download(ticker, period="1d")['Adj Close'].iloc[-1]
print(f"最新の株価: ¥{latest_price:.2f}")